# IDENTIFYING CITIES TO INVEST IN RE

Identify best cities to invest in real estate
    This does not apply to super expensive cities like Down town San Francisco or Manhattan
    This does not apply to very bad neightborhoods (warzones)
## CITIES REALFOCUS FACTORS   
### 1. Population growth
 Population data is retrieved from https://www.city-data.com

| City type | Population | Ideal Rate of Growth (2000-2017) | Ideal rate of growth per year afterwards
| :- | :- |:- | :- |
| Large metro cities | 1-2 Million | 15% | 1%
| Micro cities | < 250k | 30% | 15%
| Moderate | 250k-1 Million | 21.25% | 1.25

Reference: Webinar by

2. 

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import logging

In [6]:
df = {}  
df_city_vals = pd.DataFrame()

In [7]:
def parse_city_data(city_state):
    city_values = pd.DataFrame()
    cols = ['City','Population_2000', 'Population', 'pop_%', 'Median_income_2000', 'Median_income_2017', 'Income_%',
            'Median_House_2000', 'Median_House_2017', 'house_%', 
            'Crime_index_2005', 'Crime_index_2018', 'crime_%']

    city_link = get_city_address(city_state)    
    text = requests.get(city_link)
    soup = BeautifulSoup(text.content, 'html.parser')
    
    # get populataion section
    section_population = soup.find('section', {'id':'city-population'})
    # get median income and house text
    section_med_inc = soup.find('section', {'id':'median-income'})
    # get crime section
    section_crime_index = soup.find('section', {'id':'crime'})
    
    
    tag_txt_house = 'Estimated median house or condo value in 2017:'
    tag_txt_income = 'Estimated median household income in 2017:'
    tag_txt_population = 'Population in 2017:'
        
    tag = section_med_inc.find('b',text=tag_txt_house)
 
    city_values == {}
    all_fields = []
    # get population data 
    population_fields = get_population(tag_txt_population, section_population)
    # find and update the income values
    income_fields = get_city_values(tag_txt_income, section_med_inc)
    # find and update the median house values
    house_fields = get_city_values(tag_txt_house, section_med_inc)
    # find crimes index data
    crime_index_fields = get_crime_index(section_crime_index)

    all_fields.append(city_state)
    all_fields.extend(population_fields)
    all_fields.extend(income_fields)
    all_fields.extend(house_fields)
    all_fields.extend(crime_index_fields)
    all_fields
         
    global df_city_vals
    df_city_vals = df_city_vals.append(pd.DataFrame([all_fields], 
                                                    columns=cols),ignore_index = True)   

    
def get_city_values(srch_txt, section):
    tag = section.find('b',text=srch_txt)   
    latest_element = tag.next_sibling
    compare_element = tag.findNext('b').next_sibling
    compare_col = convert_to_num(compare_element)
    latest_col = convert_to_num(latest_element)    
    pct_diff =  get_pct(latest_col, compare_col)
    
    return [int(compare_col), int(latest_col), float(pct_diff)]

def get_population(srch_txt, section):
#     print(section)
    tag = section.find('b',text=srch_txt) 
    latest_col_txt = tag.next_sibling
    latest_col_txt = latest_col_txt.split('(')[0];
    latest_col = convert_to_num(latest_col_txt)    

    pct_diff = tag.findNext('b').next_sibling.split('%')[0]
    pct_diff = float(pct_diff)
    
    # poulation of latest year and % diff is provided, need to calculate what was the population
    compare_popln = latest_col/(1 + pct_diff/100);

#     compare_col = convert_to_num(compare_element)
#     latest_col = convert_to_num(latest_element)    
#     pct_diff =  get_pct(latest_col, compare_col)
    
    return [int(compare_popln), int(latest_col), float(pct_diff)]

def get_crime_index(section_crime):
    # if no crime data found for the city
    if section_crime == None:
        return [None, None, None]
    cr = section_crime.find_all('tr', {"class": "nosort"})[0]
    crimes = []
    for idx, val in enumerate(cr.find_all('td')):
        if idx != 0:
            crime = val.text.strip()
            crimes.append(float(crime))
    cr_pct_dff = get_pct(crimes[0], crimes[1])            
    return [crimes[0], crimes[-1], cr_pct_dff] 

def convert_to_num(text):
    tex = text.replace(',', '')
    return int(re.search('[0-9]+',tex).group(0))

def get_pct(num1, num2):
    return round((num1/num2  - 1) * 100, 2)

def get_city_address(City_link):
    city_data_address = 'http://www.city-data.com'
    return city_data_address + '/city/' + City_link + '.html'

       

In [8]:
cities_data = 'data/large_market_database.csv'
df = pd.read_csv(cities_data)

cities = []
for city in df['Location']:
    #   concatenate city and state with '-', replace empty space with '-' and ' with -
    #   O'Fallon-Missouri to O-Fallon-Missouri
    cy = city.replace(' , ', '-').replace(' ','-').replace("'", '-').replace(',-','-').replace('--', '-')
    cities.append(cy)
df['Cities_ready'] = cities    

# drop cities containing "balance" 
# like"Augusta-Richmond-County-consolidated-government-(balance)-Georgia"
df_cy = df[~df.Cities_ready.str.contains("balance")]

citis_ser = df_cy.iloc[0:500]['Cities_ready']
# print(df1.where('Cities_ready' == 'Miami-Gardens-Florida'))

# #for testing
# data = np.array(['Miami-Gardens-Florida', "Flint-Michigan"])  
# citis_ser = pd.Series(data)

# parse_city_data('Miami-Gardens-Florida')        
for city in citis_ser:
    print('current: ' + city)
    try:
        parse_city_data(city)
    except Exception as e:
        print('Data not found for: ' + city)
        pass
    
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_city_vals)

current: Flint-Michigan
current: Detroit-Michigan
current: Youngstown-Ohio
current: Gary-Indiana
current: Reading-Pennsylvania
current: Trenton-New-Jersey
current: Dayton-Ohio
current: Cleveland-Ohio
current: Rochester-New-York
current: Muncie-Indiana
current: Lansing-Michigan
current: Hammond-Indiana
current: Kansas-City-Kansas
current: Canton-Ohio
current: South-Bend-Indiana
current: Buffalo-New-York
current: Memphis-Tennessee
current: Jackson-Mississippi
current: Akron-Ohio
current: Birmingham-Alabama
current: Toledo-Ohio
current: Largo-Florida
current: Tamarac-Florida
current: Syracuse-New-York
current: Sunrise-Florida
current: Rockford-Illinois
current: Deerfield-Beach-Florida
current: Baytown-Texas
current: Mesquite-Texas
current: Warren-Michigan
current: Deltona-Florida
current: Evansville-Indiana
current: Decatur-Illinois
current: Lorain-Ohio
current: Brownsville-Texas
current: Beaumont-Texas
current: Wichita-Falls-Texas
current: Warner-Robins-Georgia
current: Abilene-Texas
cur

current: Chesapeake-Virginia
current: Menifee-California
Data not found for: Menifee-California
current: Murfreesboro-Tennessee
current: Ames-Iowa
current: Gilbert-Arizona
current: Suffolk-Virginia
current: Columbia-Missouri
current: Phoenix-Arizona
current: Lafayette-Louisiana
current: Plano-Texas
current: Huntsville-Alabama
current: Madison-Wisconsin
current: San-Bernardino-California
current: Newark-New-Jersey
current: Waukesha-Wisconsin
current: West-Jordan-Utah
current: Georgetown-Texas
current: Raleigh-North-Carolina
current: Columbia-Maryland
current: Kennewick-Washington
current: Farmington-Hills-Michigan
current: Antioch-California
current: McKinney-Texas
current: Jupiter-Florida
current: Sioux-Falls-South-Dakota
current: Las-Vegas-Nevada
current: Merced-California
current: Cheyenne-Wyoming
current: Bakersfield-California
current: Cranston-Rhode-Island
current: Visalia-California
current: Edmond-Oklahoma
current: Richmond-Virginia
current: Brockton-Massachusetts
current: Westm